# Imports de base

In [1]:
#importation des librairies
import pandas as pd #matrice
import sklearn as sk #Machine Learning
import numpy as np

from datetime import datetime
import time

from math import *
import statistics

from joblib import dump, load

#from blog saladtomatonion.com
class Timer(object):
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, *args, **kwargs):
        self.stop()
        
    def start(self):
        if hasattr(self, 'interval'):
            del self.interval
        self.start_time = time.time()
    
    def stop(self):
        if hasattr(self, 'start_time'):
            self.interval = time.time() - self.start_time
            del self.start_time
            
def formatDFsansOthers(df):
    #mise en minuscule
    df["Entité"] = df["Entité"].str.lower()
    df["Compte"] = df["Compte"].str.lower()
    df["Fournisseur identification"] = df["Fournisseur identification"].str.lower()
    df["Cd_Scat"] = df["Cd_Scat"].str.lower()
    df["Fournisseur"] = df["Fournisseur"].str.lower()
    df["Family Stream"] = df["Family Stream"].str.lower()
    df["Category Stream"] = df["Category Stream"].str.lower()
    df["Sub Category Stream"] = df["Sub Category Stream"].str.lower()
    
    #suppression des lignes non voulues pour l'apprentissage :
    #les others qui ne sont pas des "transactions fees"
    #EN COMMENTAIRE POUR L'APPLICATION AU RUN et POUR POUVOIR APPRENDRE LES OTHERS EGALEMENT
    indexNames = df[ (df['Family Stream'] == 'others') & (df['Category Stream'] != 'transaction fees')].index
    df.drop(indexNames , inplace=True)

    #remplis les vide avec un tiret "-"
    df.fillna("-", inplace=True)

    #aggrega
    #df["AcléMin"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]
    #df["AcléMinE"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]

    df["AcléEt"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléEtSansE"]=df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléTout"]=df["Entité"]+';'+df["CodePays"]+';'+df["Compte"]+';'+df["Sous_catégorie_de_dépenses"]+';'+df["Catégorie_de_dépenses"]+';'+df["Famille_de_dépenses"]+';'+df["Fournisseur"]+';'+df["Fournisseur identification"]

    df["Acible"]=df["Family Stream"]+" ; "+df["Category Stream"]+" ; "+df["Sub Category Stream"]
    
    return df

def formatDFsansOthersToLearn(df):
    #mise en minuscule
    df["Entité"] = df["Entité"].str.lower()
    df["Compte"] = df["Compte"].str.lower()
    df["Fournisseur identification"] = df["Fournisseur identification"].str.lower()
    df["Cd_Scat"] = df["Cd_Scat"].str.lower()
    df["Fournisseur"] = df["Fournisseur"].str.lower()
    #df["Family Stream"] = df["Family Stream"].str.lower()
    #df["Category Stream"] = df["Category Stream"].str.lower()
    #df["Sub Category Stream"] = df["Sub Category Stream"].str.lower()
    
    #suppression des lignes non voulues pour l'apprentissage :
    #les others qui ne sont pas des "transactions fees"
    #EN COMMENTAIRE POUR L'APPLICATION AU RUN et POUR POUVOIR APPRENDRE LES OTHERS EGALEMENT
    #indexNames = df[ (df['Family Stream'] == 'others') & (df['Category Stream'] != 'transaction fees')].index
    #df.drop(indexNames , inplace=True)

    #remplis les vide avec un tiret "-"
    df.fillna("-", inplace=True)

    #aggrega
    #df["AcléMin"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]
    #df["AcléMinE"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]

    df["AcléEt"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléEtSansE"]=df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléTout"]=df["Entité"]+';'+df["CodePays"]+';'+df["Compte"]+';'+df["Sous_catégorie_de_dépenses"]+';'+df["Catégorie_de_dépenses"]+';'+df["Famille_de_dépenses"]+';'+df["Fournisseur"]+';'+df["Fournisseur identification"]

    #df["Acible"]=df["Family Stream"]+" ; "+df["Category Stream"]+" ; "+df["Sub Category Stream"]
    
    return df

## Fichier contenant les lignes sur lesquelles apprendre

In [2]:
dfTL = pd.read_excel('./Lignes à apprendre.xlsx')

## Fichier contenant les lignes à déterminer

In [8]:
dfTP = pd.read_excel('./Lignes à prédire.xlsx')

# Traitements préliminaires sur les fichiers chargés

In [4]:
#Mets les données au bon format et les enregistre sous un format facilement récupérable (dump)
dfToLearn = dfTL.copy()
dfToPredict = dfTP.copy()

dfToLearn = formatDFsansOthers(dfToLearn)
dump(dfToLearn, 'dfToLearn.dump')

dfToPredict = formatDFsansOthersToLearn(dfToPredict)
dump(dfToPredict, 'dfToPredict.dump')

['dfToPredict.dump']

In [9]:
#lit le dump des données à partir desquelles apprendre (si existant, plutôt que de traiter le fichier excel à chaque fois)

#enlever le commentaire en enlevant le dièse "#" de la ligne suivante pour charger le dump existant
dfToLearn = load('dfToLearn.dump')

In [3]:
#lit le dump des données à prédire (si existant, plutôt que de traiter le fichier excel à chaque fois)

#enlever le commentaire en enlevant le dièse "#" de la ligne suivante pour charger le dump existant
dfToPredict = load('dfToPredict.dump')

In [5]:
dfToLearn

,CdEnt,Entité,CodePays,Pôle_de_l_entité,Secteur Pôle,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Expense_Sub_Category,...,SommeDekEUR,Clé,Family Stream,Category Stream,Sub Category Stream,Métier v2,Country,Region,AcléEt,Acible
0,UA002,ukrsibbank,UA,RB&S,International Financial Services,USB.FBANK,means of payment / bank charges,fbank,Frais Bancaires,Bank Charges,...,23955.72090,UA002_USB.FBANK_USB.90000000124,banking services,credit cards (incl processing),-,International Retail Banking,Ukraine,EMEA,ukrsibbank ; means of payment / bank charges ;...,banking services ; credit cards (incl processi...
1,DE021,consorsbank,DE,RB&S,Domestic Markets,CONSORS.653206,operation expenses kordoba,bqext0,z-Prestations bancaires externalisées - autres,z-Outsourced banking services - others,...,16374.43823,DE021_CONSORS.653206_CONSORS.24596,banking services,banking software & services,-,Personal Investors,Germany,EMEA,consorsbank ; operation expenses kordoba ; fis...,banking services ; banking software & services...
2,DE021,consorsbank,DE,RB&S,Domestic Markets,CONSORS.653206,operation expenses kordoba,bqext0,z-Prestations bancaires externalisées - autres,z-Outsourced banking services - others,...,14859.60519,DE021_CONSORS.653206_CONSORS.24596,banking services,banking software & services,-,Personal Investors,Germany,EMEA,consorsbank ; operation expenses kordoba ; fis...,banking services ; banking software & services...
3,UA002,ukrsibbank,UA,RB&S,International Financial Services,USB.FBANK,means of payment / bank charges,fbank,Frais Bancaires,Bank Charges,...,12548.08372,UA002_USB.FBANK_USB.90000000124,banking services,credit cards (incl processing),-,International Retail Banking,Ukraine,EMEA,ukrsibbank ; means of payment / bank charges ;...,banking services ; credit cards (incl processi...
4,FR513,bddf - entreprises,FR,RB&S,Domestic Markets,OCAPP.241392,bank cards 24 months,carte,Cartes bancaires,Cartes bancaires,...,11814.61015,FR513_OCAPP.241392_OCAPP.1001049,banking services,credit cards (incl processing),-,French Retail Banking,France,France,bddf - entreprises ; bank cards 24 months ; ge...,banking services ; credit cards (incl processi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823776,SE65826,sevendays,-,RB,Retail Banking,MSDYNAX_EB.825460,recurring telephony fees,???,Non affecté à une famille précise,Not affected to a precise family,...,15.24251,SE65826_MSDYNAX_EB.825460_MSDYNAX_EB.10200,technology,telco,-,Personal Finance,Sweden,EMEA,sevendays ; recurring telephony fees ; sinch s...,technology ; telco ; -
823777,SE65826,sevendays,-,RB,Retail Banking,MSDYNAX_EB.825460,recurring telephony fees,???,Non affecté à une famille précise,Not affected to a precise family,...,1.35724,SE65826_MSDYNAX_EB.825460_MSDYNAX_EB.10249,technology,telco,-,Personal Finance,Sweden,EMEA,sevendays ; recurring telephony fees ; tellus ...,technology ; telco ; -
823778,SE65826,sevendays,-,RB,Retail Banking,MSDYNAX_EB.825510,other it costs,???,Non affecté à une famille précise,Not affected to a precise family,...,5.02170,SE65826_MSDYNAX_EB.825510_MSDYNAX_EB.10144,technology,it services,-,Personal Finance,Sweden,EMEA,sevendays ; other it costs ; mybw office manag...,technology ; it services ; -
823779,SE65826,sevendays,-,RB,Retail Banking,MSDYNAX_EB.831090,misc. office supplies,fourn,Fournitures de bureau,Office supplies,...,0.15345,SE65826_MSDYNAX_EB.831090_MSDYNAX_EB.10144,technology,software,-,Personal Finance,Sweden,EMEA,sevendays ; misc. office supplies ; mybw offic...,technology ; software ; -


In [6]:
dfToPredict

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Expense_Sub_Category,Cd_Cat,Catégorie_de_dépenses,...,Key,Family Stream,Category Stream,Sub Category Stream,Pôle_de_l_entité,Secteur Pôle,Métier v2,Country,Region,AcléEt
0,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,DE22872_BPLG_DE.6391540000_BPLG_DE.FR58809096480,-,-,-,RB,Retail Banking,Leasing Solutions,Germany,EMEA,bnpp lease group cologne branch ; personalbesc...
1,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,DE22872_BPLG_DE.6391540000_BPLG_DE.DE301275346,-,-,-,RB,Retail Banking,Leasing Solutions,Germany,EMEA,bnpp lease group cologne branch ; personalbesc...
2,DE22152,arval deutschland gmbh,DE,ARVAL_DE.XXX,verhan,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,DE22152_ARVAL_DE.XXX_ARVAL_DE.C1,-,-,-,RB,Retail Banking,Arval,Germany,EMEA,arval deutschland gmbh ; verhan ; greenval ins...
3,CN23534,bnpp (china) ltd,CN,SAP_LIGHT.SAP|676300,other - other,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,CN23534_SAP_LIGHT.SAP|676300_SAP_LIGHT.NA,-,-,-,CIB,Corporate & Institutional Banking,CIB (excl BPSS),China,APAC,bnpp (china) ltd ; other - other ; xxx ; ??? ;...
4,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391700000,sachversicherungen,assu0,z-Assurances - autres,z-Insurance - others,ASSU0,Z-Assurances - autres,...,DE22872_BPLG_DE.6391700000_BPLG_DE.LU16602944,-,-,-,RB,Retail Banking,Leasing Solutions,Germany,EMEA,bnpp lease group cologne branch ; sachversiche...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33264,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.51909545,activos fijos menores,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,CO44866_CARDIF_CO.51909545_CARDIF_CO.8002374121,-,-,-,IPS,Investment & Protection Services,BNP Paribas Cardif,Colombia,Americas,cardif colombia seguros generales sa ; activos...
33265,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.5190959924,consultas desempleo experian,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,CO44866_CARDIF_CO.5190959924_CARDIF_CO.9004226148,-,-,-,IPS,Investment & Protection Services,BNP Paribas Cardif,Colombia,Americas,cardif colombia seguros generales sa ; consult...
33266,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.52959531,gastos no deducibles tc,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,CO44866_CARDIF_CO.52959531_CARDIF_CO.9002004353,-,-,-,IPS,Investment & Protection Services,BNP Paribas Cardif,Colombia,Americas,cardif colombia seguros generales sa ; gastos ...
33267,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.52959533,gastos de ejercicios anteriores personal,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,CO44866_CARDIF_CO.52959533_CARDIF_CO.8600073361,-,-,-,IPS,Investment & Protection Services,BNP Paribas Cardif,Colombia,Americas,cardif colombia seguros generales sa ; gastos ...


# Machine Learning

In [4]:
#début ML
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.impute import KNNImputer as KNNI
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neural_network import MLPClassifier

from nltk.corpus import stopwords

# code d'APPRENTISSAGE ET RESTITUTION SIMPLE

In [7]:
def definiVect(X):
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                         min_df = 3, stop_words = 'english', sublinear_tf=True)
    
    print(str(datetime.now())+" - __start__")
    with Timer() as timer:
        vectRun = vectorizer.fit(X)
    print(str(datetime.now())+' - temps de traitement du fitting : '+str(timer.interval))
    
    dump(vectRun, 'vectRun.modele')
    
    return vectRun

def apprendModeles(X, y, labelDataApprises):
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                         min_df = 3, stop_words = 'english', sublinear_tf=True)
    
    print(str(datetime.now())+" - __start__")
    with Timer() as timer:
        vectRun = vectorizer.fit(X)
    print(str(datetime.now())+' - temps de traitement du fitting : '+str(timer.interval))
    with Timer() as timer:
        X_RUN = vectRun.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))
    dump(vectRun, 'vectRun.modele')
    
    modelLRRun = LogisticRegression(solver='sag', C=60, max_iter=1200)
    modelPACRun = PassiveAggressiveClassifier(C=0.5, max_iter=800)
    modelSGDCRunH = SGDClassifier(alpha=2e-8, loss='hinge', max_iter=400)
    modelSGDCRunL = SGDClassifier(alpha=1e-8, loss='log', max_iter=400)
    modelSGDCRunMH = SGDClassifier(alpha=1e-7, loss='modified_huber', max_iter=400)
    modelLSVCRunH = LinearSVC(max_iter = 1200, loss='hinge', C=40)
    modelLSVCRunSH = LinearSVC(max_iter = 1200, loss='squared_hinge', C=7.5)
    
    with Timer() as timer:
        modelLRRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLRRun : '+str(timer.interval))
    dump(modelLRRun, 'modeleLR_'+labelDataApprises+'.modele')
    
    with Timer() as timer:
        modelPACRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelPACRun : '+str(timer.interval))
    dump(modelPACRun, 'modelePAC_'+labelDataApprises+'.modele')
    
    with Timer() as timer:
        modelSGDCRunH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelSGDCRunH : '+str(timer.interval))
    dump(modelSGDCRunH, 'modeleSGD_H_'+labelDataApprises+'.modele')
    with Timer() as timer:
        modelSGDCRunL.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelSGDCRunL : '+str(timer.interval))
    dump(modelSGDCRunL, 'modeleSGD_L_'+labelDataApprises+'.modele')
    with Timer() as timer:
        modelSGDCRunMH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelSGDCRunMH : '+str(timer.interval))
    dump(modelSGDCRunMH, 'modeleSGD_MH_'+labelDataApprises+'.modele')
    
    with Timer() as timer:
        modelLSVCRunH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLSVCRunH : '+str(timer.interval))
    dump(modelLSVCRunH, 'modeleLSVC_H_'+labelDataApprises+'.modele')
    with Timer() as timer:
        modelLSVCRunSH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLSVCRunSH : '+str(timer.interval))
    dump(modelLSVCRunSH, 'modeleLSVC_SH_'+labelDataApprises+'.modele')
    
    return vectRun, modelLRRun, modelPACRun, modelSGDCRunH, modelSGDCRunL, modelSGDCRunMH, modelLSVCRunH, modelLSVCRunSH

def preditStream(vecteur, X, modelLR, modelPAC, modelSGDC_H, modelSGDC_L, modelSGDC_MH, modelLSVC_H, modelLSVC_SH):
    with Timer() as timer:
        X_to_predict = vecteur.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))
    
    with Timer() as timer:
        predictedLR = modelLR.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLR : '+str(timer.interval))
    with Timer() as timer:
        predictedPAC = modelPAC.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelPAC : '+str(timer.interval))
    with Timer() as timer:
        predictedSGD_H = modelSGDC_H.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelSGDC_H : '+str(timer.interval))
    with Timer() as timer:
        predictedSGD_L = modelSGDC_L.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelSGDC_L : '+str(timer.interval))
    with Timer() as timer:
        predictedSGD_MH = modelSGDC_MH.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelSGDC_MH : '+str(timer.interval))
    with Timer() as timer:
        predictedSVC_H = modelLSVC_H.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLSVC_H : '+str(timer.interval))
    with Timer() as timer:
        predictedSVC_SH = modelLSVC_SH.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLSVC_SH : '+str(timer.interval))
    
    return predictedLR, predictedPAC, predictedSGD_H, predictedSGD_L, predictedSGD_MH, predictedSVC_H, predictedSVC_SH

# Apprentissage

In [7]:
vectRunToLearn, modelLRRunToLearn, modelPACRunToLearn, modelSGDCRunHToLearn, modelSGDCRunLToLearn, modelSGDCRunMHToLearn, modelLSVCRunHToLearn, modelLSVCRunSHToLearn = \
    apprendModeles(dfToLearn["AcléEt"], dfToLearn["Acible"], 'ToLearn')

2021-09-27 21:37:10.412526 - __start__
2021-09-27 21:37:51.642867 - temps de traitement du fitting : 41.2303409576416
2021-09-27 21:38:27.923368 - temps de traitement du transform : 36.2805016040802
2021-09-27 22:01:42.531518 - temps de traitement du fit de modelLRRun : 1381.0147604942322
2021-09-27 22:05:08.800911 - temps de traitement du fit de modelPACRun : 205.84066247940063
2021-09-27 22:10:18.274993 - temps de traitement du fit de modelSGDCRunH : 309.04079127311707
2021-09-27 22:16:48.333971 - temps de traitement du fit de modelSGDCRunL : 389.56243324279785
2021-09-27 22:23:35.118096 - temps de traitement du fit de modelSGDCRunMH : 406.34349036216736


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-09-27 22:58:40.347049 - temps de traitement du fit de modelLSVCRunH : 2104.783542394638
2021-09-27 23:17:35.874245 - temps de traitement du fit de modelLSVCRunSH : 1135.125961303711


### chargement de modèles enregistrés (déjà appris)

In [11]:
#si existant, plutôt que de refaire un apprentissage à chaque fois : 
#évite également d'avoir besoin de charger le fichier contenant les données à partir desquelles apprendre

#enlever les commentaires en enlevant le dièse "#" des lignes suivantes pour charger les modèles existant
modelLRRunToLearn = load('modeleLR_ToLearn.modele')
modelPACRunToLearn = load('modelePAC_ToLearn.modele')
modelSGDCRunHToLearn = load('modeleSGD_H_ToLearn.modele')
modelSGDCRunLToLearn = load('modeleSGD_L_ToLearn.modele')
modelSGDCRunMHToLearn = load('modeleSGD_MH_ToLearn.modele')
modelLSVCRunHToLearn = load('modeleLSVC_H_ToLearn.modele')
modelLSVCRunSHToLearn = load('modeleLSVC_SH_ToLearn.modele')
vectRunToLearn = load('vectRun.modele')

# Prédiction

In [12]:
dfToPredict["LR_predToLearn"], dfToPredict["PAC_predToLearn"], dfToPredict["SGD_H_predToLearn"], dfToPredict["SGD_L_predToLearn"], dfToPredict["SGD_MH_predToLearn"], dfToPredict["SVC_H_predToLearn"], dfToPredict["SVC_SH_predToLearn"] = \
    preditStream(vectRunToLearn, dfToPredict["AcléEt"], modelLRRunToLearn, modelPACRunToLearn, modelSGDCRunHToLearn, modelSGDCRunLToLearn, modelSGDCRunMHToLearn, modelLSVCRunHToLearn, modelLSVCRunSHToLearn)

2022-03-24 15:40:03.914562 - temps de traitement du transform : 3.2115111351013184
2022-03-24 15:40:04.713756 - temps de prediction de modelLR : 0.7991940975189209
2022-03-24 15:40:05.568617 - temps de prediction de modelPAC : 0.8548614978790283
2022-03-24 15:40:06.291130 - temps de prediction de modelSGDC_H : 0.72251296043396
2022-03-24 15:40:06.977154 - temps de prediction de modelSGDC_L : 0.6860232353210449
2022-03-24 15:40:07.675292 - temps de prediction de modelSGDC_MH : 0.6981387138366699
2022-03-24 15:40:07.820051 - temps de prediction de modelLSVC_H : 0.144758939743042
2022-03-24 15:40:07.997567 - temps de prediction de modelLSVC_SH : 0.17751598358154297


In [13]:
dfToPredict

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Expense_Sub_Category,Cd_Cat,Catégorie_de_dépenses,...,Country,Region,AcléEt,LR_predToLearn,PAC_predToLearn,SGD_H_predToLearn,SGD_L_predToLearn,SGD_MH_predToLearn,SVC_H_predToLearn,SVC_SH_predToLearn
0,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,Germany,EMEA,bnpp lease group cologne branch ; personalbesc...,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events
1,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,Germany,EMEA,bnpp lease group cologne branch ; personalbesc...,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...
2,DE22152,arval deutschland gmbh,DE,ARVAL_DE.XXX,verhan,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,Germany,EMEA,arval deutschland gmbh ; verhan ; greenval ins...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,bnp paribas group ; - ; -,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...
3,CN23534,bnpp (china) ltd,CN,SAP_LIGHT.SAP|676300,other - other,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,China,APAC,bnpp (china) ltd ; other - other ; xxx ; ??? ;...,technology ; software ; -,technology ; software ; -,technology ; software ; -,professional services ; marcom ; sponsoring,technology ; software ; -,technology ; software ; -,technology ; software ; -
4,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391700000,sachversicherungen,assu0,z-Assurances - autres,z-Insurance - others,ASSU0,Z-Assurances - autres,...,Germany,EMEA,bnpp lease group cologne branch ; sachversiche...,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33264,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.51909545,activos fijos menores,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,Colombia,Americas,cardif colombia seguros generales sa ; activos...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...
33265,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.5190959924,consultas desempleo experian,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,Colombia,Americas,cardif colombia seguros generales sa ; consult...,corporate services ; corporate real estate ; c...,technology ; it services ; -,corporate services ; corporate real estate ; c...,technolog

In [10]:
dfToPredict.to_excel('lignes à prédire - streamées simple.xlsx')

# Amélioration de la prédiction en croisant les résultats

In [14]:
def traiteRecursifDf(df, dico, nbLignes, nbrElemTotal, nbrParmiTotal, numCourant, numElem, numInitial, \
                     tableauVal, complementLabel):
    
    if numCourant == nbrParmiTotal:
        label = ''
        
        for i in range(0, len(tableauVal) - 1):
            label = label+str(tableauVal[i])
        
        labelPrec = label
        labelBase = str(tableauVal[0]) + str(tableauVal[len(tableauVal) - 1])
        label = label+str(tableauVal[len(tableauVal) - 1])
        
        #print(label)
        
        dfTemp = pd.DataFrame()
        if complementLabel == '':
            if nbrParmiTotal == 2:
                df[label] = [a if a==b else '-' for a,b in zip(df[tableauVal[0]],df[tableauVal[1]])]
            else:
                df[label] = [a if a==b else '-' for a,b in zip(df[labelBase],df[labelPrec])]
            
            if False:
                if nbrParmiTotal == 2:
                    df[label] = [1 if a==b else 0 for a,b in zip(df[tableauVal[0]],df[tableauVal[1]])]
                else:
                    df[label] = [a*b for a,b in zip(df[labelBase],df[labelPrec])]

                dico[label] = {}
                dico[label]['%'] = 100*sum(df[label])/nbLignes
        else:
            df[label+complementLabel]=[a*b for a,b in zip(df[str(numInitial)+complementLabel],df[label])]
            if sum(df[label]) > 0:
                dico[label]['%'+complementLabel] = 100*sum(df[label+complementLabel])/sum(df[label])
            else:
                dico[label]['%'+complementLabel] = 0
        
        if False:
            dico[label]['nbr'+complementLabel] = sum(df[label+complementLabel])

    else:
        for i in range(numElem+1, nbrElemTotal + 1):
            tableauVal.append(i)
            df, dico = traiteRecursifDf(df, dico, nbLignes, nbrElemTotal, nbrParmiTotal, numCourant+1, i, \
                                        numInitial, tableauVal, complementLabel)
            tableauVal.pop()
            
    return df, dico

def traiteDFRetourRec(dfTravail, champCible, ext):
    tableauLabels = ['LR_predToLearn', 'PAC_predToLearn',\
                    'SGD_H_predToLearn', 'SGD_L_predToLearn', 'SGD_MH_predToLearn',\
                    'SVC_H_predToLearn', 'SVC_SH_predToLearn']
    #tableauLabels = ['LogisticRegression_liblinear_'+ext,'LogisticRegression_sag_'+ext,'LogisticRegression_saga_'+ext,\
    #                 'PassiveAggressive_'+ext,'SGD_hinge_'+ext,'SGD_log_'+ext,'SGD_modified_huber_'+ext,'LinearSVC_hinge_'+ext,\
    #                 'LinearSVC_squared_hinge_'+ext]
    nbTotalElem = len(tableauLabels)
    for i,label in zip([x for x in range(1, nbTotalElem + 1)],tableauLabels):
        dfTravail[i]=dfTravail[label]
    
    nbLignes = dfTravail.shape[0]
    
    dicoTemp={}
    if False:
        dicoTemp['nbL'] = {}
        dicoTemp['nbL']['nbr'] = nbLignes
        dicoTemp['nbL']['%'] = 0
        dicoTemp['nbL']['nbr OK'] = 0
        dicoTemp['nbL']['% OK'] = 0

        #si chacun est pareil que cible, résultat attendu
        for i in range(1, nbTotalElem + 1):
            dfTravail[str(i)+' OK'] = [1 if a==b else 0 for a,b in zip(dfTravail[i],dfTravail[champCible])]
            dicoTemp[str(i)] = {}
            dicoTemp[str(i)]['nbr'] = nbLignes
            dicoTemp[str(i)]['%'] = 100
            dicoTemp[str(i)]['nbr OK'] = sum(dfTravail[str(i)+' OK'])
            dicoTemp[str(i)]['% OK'] = 100*sum(dfTravail[str(i)+' OK'])/nbLignes
    
    #lance le travail récursif
    #considère nbrElemPrisEnCompte parmi le nombre total d'éléments
    #soit 2 parmi n, puis 3 parmi n,...
    for nbrElemPrisEnCompte in range(2, len(tableauLabels) + 1):
        for i in range(1, len(tableauLabels) + 1):
            dfTravail, dicoTemp = traiteRecursifDf(dfTravail, dicoTemp, nbLignes, len(tableauLabels), \
                                                   nbrElemPrisEnCompte, 1, i, i, [i], '')
            if False:
                dfTravail, dicoTemp = traiteRecursifDf(dfTravail, dicoTemp, nbLignes, len(tableauLabels), \
                                                       nbrElemPrisEnCompte, 1, i, i, [i], ' OK')
    
    dfResultatsT = pd.DataFrame.from_dict(dicoTemp)
    
    return dfTravail, dfResultatsT

In [15]:
with Timer() as timerTout:
    dfCompletRec, dfResultatsTRec = traiteDFRetourRec(dfToPredict, '', '')
    #a=1
print()
print(str(datetime.now())+' - temps de generation des possibles : '+str(timerTout.interval))


2022-03-24 15:47:43.322078 - temps de generation des possibles : 4.017445087432861


In [16]:
dfCompletRec

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Expense_Sub_Category,Cd_Cat,Catégorie_de_dépenses,...,24567,34567,123456,123457,123467,123567,124567,134567,234567,1234567
0,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events
1,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,-,professional services ; recruitment fees ; gen...,-,-,-,-,-,-,-,-
2,DE22152,arval deutschland gmbh,DE,ARVAL_DE.XXX,verhan,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,-,-,-,-,-,corporate services ; corporate real estate ; c...,-,-,-,-
3,CN23534,bnpp (china) ltd,CN,SAP_LIGHT.SAP|676300,other - other,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,-,-,-,-,-,technology ; software ; -,-,-,-,-
4,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391700000,sachversicherungen,assu0,z-Assurances - autres,z-Insurance - others,ASSU0,Z-Assurances - autres,...,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33264,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.51909545,activos fijos menores,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...
33265,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.5190959924,consultas desempleo experian,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,-,-,-,-,-,-,-,-,-,-
33266,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.52959531,gastos no deducibles tc,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -,bnp paribas group ; - ; -
33267,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.52959533,gastos de ejercicios anteriores personal,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,corporate services ; general services ; social...,-,-,-,-,-,corporate services ; general services ; social...,-,-,-


In [18]:
dfResultatsTRec

""


In [42]:
def streamsIdentiques(df):
    dfToPredict['14']=[a if a==b else "-" for a,b in zip(dfToPredict.iloc[:,34+i])]
    for i in range(0,14):
        dfToPredict.iloc[:,34+i]

0        corporate services ; corporate real estate ; c...
1        corporate services ; general services ; social...
2        corporate services ; corporate real estate ; m...
3        banking services ; credit cards (incl processi...
4                corporate services ; travel ; restaurants
                               ...                        
12745    corporate services ; corporate real estate ; f...
12746    corporate services ; corporate real estate ; c...
12747    corporate services ; corporate real estate ; c...
12748      corporate services ; general services ; postage
12749    corporate services ; corporate real estate ; c...
Name: LR_predKhalil, Length: 12750, dtype: object
0        corporate services ; corporate real estate ; c...
1        corporate services ; general services ; social...
2        corporate services ; corporate real estate ; m...
3        banking services ; credit cards (incl processi...
4                corporate services ; travel ; restaurants
      

In [19]:
dfCompletRec.to_excel('lignes à prédire - streamées croisées.xlsx')

In [29]:
len(dfCompletRec[dfCompletRec['1234567'] != "-"]) / dfCompletRec.shape[0]

0.6910637530433738

In [28]:
dfCompletRec.shape[0]

33269